# Import libraries

In [20]:
# Fix randomness and hide warnings
RND = False
if not RND:
    seed = 76998669

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
if not RND:
    os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
if not RND:
    np.random.seed(seed)

import logging

import random
if not RND:
    random.seed(seed)

In [21]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
from keras.preprocessing.image import ImageDataGenerator

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
if not RND:
    tf.random.set_seed(seed)
    tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [22]:
# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

# Load data

In [23]:
# download clean dataset
!wget https://storage.googleapis.com/storage.barbiero.dev/public_data_no_meme.npz

# load dataset
dataset = np.load('public_data_no_meme.npz', allow_pickle=True)
keys = list(dataset.keys())
images = np.array(dataset[keys[0]])
labels = np.array(dataset[keys[1]])

labels_map = {0: "healthy", 1: "unhealthy"}
labels_rev_map = {"healthy": 0, "unhealthy": 1}
labels = np.array([labels_rev_map[label] for label in labels])

--2023-11-06 15:57:49--  https://storage.googleapis.com/storage.barbiero.dev/public_data_no_meme.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf, 2607:f8b0:4023:c0b::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553413073 (528M) [application/octet-stream]
Saving to: ‘public_data_no_meme.npz.1’

public_data_no_meme 100%[===================>] 527.78M  26.6MB/s    in 21s     

2023-11-06 15:58:11 (25.1 MB/s) - ‘public_data_no_meme.npz.1’ saved [553413073/553413073]



## Split data

In [24]:
# Split the dataset into a combined training and validation set, and a separate test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    images,
    labels,
    test_size = int(0.15 * len(images)),
    **({"random_state":seed} if not RND else {}),
    stratify = labels
)

# Further split the combined training and validation set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size = int(0.15 * len(images)),
    **({"random_state":seed} if not RND else {}),
    stratify = y_train_val
)

In [27]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

gen_images = 1 # Number of images that has to be generated
for img in datagen.flow(X_train,y_train,batch_size = 1):
  if gen_images <= 0:
    break
  gen_images -= 1
  X_train = np.insert(X_train,0,img[0][0],axis=0)
  y_train = np.insert(y_train,0,img[1][0],axis=0)



[[[136.76894  140.32086  120.35193 ]
  [154.01665  156.19038  142.10973 ]
  [156.39185  157.21712  147.42624 ]
  ...
  [119.       117.       102.      ]
  [119.       117.       102.      ]
  [119.       117.       102.      ]]

 [[148.08308  150.69229  132.31007 ]
  [157.00441  157.17299  145.09341 ]
  [159.99237  160.03044  151.71364 ]
  ...
  [119.       117.       102.      ]
  [119.       117.       102.      ]
  [119.       117.       102.      ]]

 [[143.48462  144.20837  127.718666]
  [152.95271  152.95271  141.65709 ]
  [162.86713  162.76665  155.31047 ]
  ...
  [119.       117.       102.      ]
  [119.       117.       102.      ]
  [119.       117.       102.      ]]

 ...

 [[ 45.338524  59.507786  24.84631 ]
  [ 46.906357  61.906357  28.906359]
  [ 45.891876  60.891876  27.927917]
  ...
  [ 69.27097  101.922585  33.61936 ]
  [ 72.353676 104.83402   36.87332 ]
  [ 75.43637  107.74546   40.127277]]

 [[ 45.68836   60.03254   25.7209  ]
  [ 46.73144   61.73144   28.731441]


## Inspect data

In [ ]:
# Print the shapes of the resulting sets
print('Training set shape:\t',X_train.shape, y_train.shape)
print('Validation set shape:\t',X_val.shape, y_val.shape)
print('Test set shape:\t\t',X_test.shape, y_test.shape)

## Process data

In [ ]:
# Normalize data to the range [0, 1]
X_train = X_train.astype("float32")/255.
X_val = X_val.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [ ]:
# Display the count of occurrences of target classes in the training-validation dataset
print('Counting occurrences of target classes:')
print(pd.DataFrame(y_train, columns=['class'])['class'].value_counts())

In [ ]:
# Convert labels to categorical format using one-hot encoding
y_train = tfk.utils.to_categorical(y_train,len(np.unique(y_train)))
y_val = tfk.utils.to_categorical(y_val,len(np.unique(y_val)))
y_test = tfk.utils.to_categorical(y_test,len(np.unique(y_test)))

In [ ]:
print('Categorical label:', y_train[0])           # Display the categorical label
print('"Default" label:', np.argmax(y_train[0]))   # Display the equivalent numeric label

# Model

In [ ]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 128                # Batch size for training
epochs = 200                     # Number of training epochs

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

## Build model

In [ ]:
# Model Function
def apple_elixir_model(input_shape, output_shape):

  preprocessing = tfk.Sequential([
        tfkl.RandomBrightness(0.2, value_range=(0,1)),
        tfkl.RandomTranslation(0.2,0.2),
        tfkl.RandomZoom(0.2),
        tfkl.RandomFlip("horizontal"),
        tfkl.RandomFlip("vertical"),
    ], name='preprocessing')

  # Build the neural network layer by layer
  input_layer = tfkl.Input(shape=input_shape, name='Input')

  preprocessing = preprocessing(input_layer)

  x = tfkl.Conv2D(filters=16, kernel_size=3,activation = 'relu')(preprocessing)
  x = tfkl.MaxPooling2D()(x)

  x = tfkl.Conv2D(filters=32,kernel_size=3,activation='relu')(preprocessing)
  x = tfkl.Conv2D(filters=32,kernel_size=3,activation='relu')(x)
  x = tfkl.MaxPooling2D()(x)

  x1 = tfkl.Conv2D(filters=32, kernel_size=3,padding='same',activation = 'relu')(x)
  x2 = tfkl.Conv2D(filters=32, kernel_size=3,padding='same', activation = 'relu')(x1)

  x = tfkl.Add()([x,x2])
  x = tfkl.ReLU()(x)
  x = tfkl.MaxPooling2D()(x)

  x = tfkl.Conv2D(filters=64,kernel_size=3,activation='relu')(x)
  x = tfkl.MaxPooling2D()(x)


  x1 = tfkl.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu')(x)
  x2 = tfkl.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu')(x1)

  x = tfkl.Add()([x,x2])
  x = tfkl.ReLU()(x)
  x = tfkl.MaxPooling2D()(x)

  x = tfkl.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu')(x)
  x = tfkl.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu')(x)
  x = tfkl.GlobalAveragePooling2D()(x)

  output_layer = tfkl.Dense(units=output_shape ,activation='softmax',name='Output')(x)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=input_layer, outputs=output_layer)

  # Compile the model
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Nadam(weight_decay=5e-4), metrics=['accuracy'])

  return model

In [ ]:
model = apple_elixir_model(input_shape, output_shape)

# Print the model summary and plot the model architecture
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

## Train model

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    mode='max',
    restore_best_weights=True)

lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',     # Metric to monitor (validation mean squared error in this case)
    patience=7,  # Number of epochs with no improvement after which learning rate will be reduced
    factor=0.90,          # Factor by which the learning rate will be reduced (0.999 in this case)
    mode='max',            # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
    min_lr=1e-7            # Minimum learning rate
)


callbacks = [early_stopping,lr_scheduler]

# Train the model and save its history
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=500,
    validation_data=(X_val, y_val),
    callbacks=callbacks
).history

# Save the trained model
model.save('CHANGE_THIS_NAME')

In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

test_predictions = model.predict(X_test, verbose=0)
test_predictions = np.argmax(test_predictions, axis=-1)
test_gt = np.argmax(y_test, axis=-1)
test_accuracy = accuracy_score(test_gt, test_predictions)
print(f'Accuracy Score over the Test Set: {round(test_accuracy, 4)}')
print()

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL

model.save('AugmentedResidual')



> larger filters

> adam optimizer


> scheduled learning rate in training seems to be effective


> lot of augmentation



> residual learning seems to be effective

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
# Compute the confusion matrix
test_predictions = model.predict(X_test, verbose=0)
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(test_predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(test_predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(test_predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(test_predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(test_predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()